In [1]:
import requests
import pandas as pd
import datetime as dt
import time

In [2]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [3]:
api = "https://api.pushshift.io/reddit/search/submission"
# after_time = 1643691600
after_time = 1641013200
final_time = 1647316800

In [4]:
subreddits = ["conservative", "liberal", "politics"]

In [5]:
start_dict = {
    "title":[],
    "score":[],
    "id":[], 
    "url":[],
    "comms_num": [],
    "created": [],
    "author":[],
    "author_flair_text":[],
    "permalink":[],
}

In [6]:
df_main = pd.DataFrame(start_dict)


In [7]:
for sub in subreddits:
    counter = after_time
    printcount = 0 
    while counter < final_time:
        try:
            request = requests.get(f'{api}?subreddit={sub}&after={counter}&size=500')
            response = request.json()
            result = response["data"]
            posts_dict = {
                "title":[],
                "score":[],
                "id":[], 
                "url":[],
                "comms_num": [],
                "created": [],
                "author":[],
                "author_flair_text":[],
                "permalink":[],
            }
            for i in result:
                posts_dict['title'].append(i['title'])
                posts_dict['score'].append(i['score'])
                posts_dict['id'].append(i['id'])
                posts_dict['url'].append(i['url'])
                posts_dict['comms_num'].append(i['num_comments'])
                posts_dict['created'].append(i['created_utc'])
                posts_dict['author'].append(i['author'])
                posts_dict['author_flair_text'].append(i['author_flair_text'])
                posts_dict['permalink'].append(i['permalink'])
            df = pd.DataFrame(posts_dict).sort_values(by=['created'], ascending=False)
            counter = df['created'].max()
            df_main = df_main.append(df)
            printcount += 1
            print(f'Success: Req #{printcount}. Lenth of df_main: {len(df_main)} -- date={get_date(counter)} -- Working on {sub}')
        except:
            print(f'ERROR: Req#{printcount} -- date={get_date(counter)} ---- {counter} -- Working on {sub}')
            time.sleep(1)

Success: Req #1. Lenth of df_main: 100 -- date=2022-01-01 10:09:33 -- Working on conservative
Success: Req #2. Lenth of df_main: 200 -- date=2022-01-01 15:12:57 -- Working on conservative
Success: Req #3. Lenth of df_main: 300 -- date=2022-01-01 21:59:10 -- Working on conservative
Success: Req #4. Lenth of df_main: 400 -- date=2022-01-02 10:16:49 -- Working on conservative
Success: Req #5. Lenth of df_main: 500 -- date=2022-01-02 14:46:24 -- Working on conservative
Success: Req #6. Lenth of df_main: 600 -- date=2022-01-02 20:11:03 -- Working on conservative
Success: Req #7. Lenth of df_main: 700 -- date=2022-01-03 07:07:54 -- Working on conservative
Success: Req #8. Lenth of df_main: 800 -- date=2022-01-03 11:09:48 -- Working on conservative
Success: Req #9. Lenth of df_main: 900 -- date=2022-01-03 14:51:13 -- Working on conservative
Success: Req #10. Lenth of df_main: 1000 -- date=2022-01-03 19:08:37 -- Working on conservative
Success: Req #11. Lenth of df_main: 1100 -- date=2022-01-0

In [12]:
_timestamp = df_main["created"].apply(get_date)
df_main = df_main.assign(timestamp=_timestamp)

In [13]:
df_main.to_csv('all_posts.csv')